### Important Notes
- Interesting to note that the 0.01 percentage training of network with only antenna_pos as input still performs reasonably well on predicting the output of shower 33. 

In [ ]:
import os
import torch
import numpy as np
import tqdm
from radioNN.data.loader import AntennaDataset

from radioNN.networks.antenna_resnet_network import AntennaNetworkResNet
from radioNN.networks.antenna_cnn_network import AntennaNetworkCNN
from radioNN.networks.antenna_fc_network import AntennaNetworkFC
from radioNN.networks.antenna_skipfc_network import AntennaNetworkSkipFC
import scipy.interpolate as intp
from radiotools.analyses import energy_fluence
import matplotlib.pyplot as plt
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
%matplotlib inline
fnt_size = 20
plt.rc('text', usetex=True)

from radioNN.process_network import NetworkProcess
radio_data_path = "/Users/denis/Desktop/BachelorThesis/memmap"
memmap_mode = "r"
if not os.path.exists(radio_data_path):
    radio_data_path = "/home/pranav/work-stuff-unsynced/radio_data"
    memmap_mode = "r"
assert os.path.exists(radio_data_path)
input_data_file = os.path.join(radio_data_path, "input_data.npy")
input_meta_file = os.path.join(radio_data_path, "meta_data.npy")
antenna_pos_file = os.path.join(
    radio_data_path, "antenna_pos_data.npy"
)
output_meta_file = os.path.join(
    radio_data_path, "output_meta_data.npy"
)
output_file = os.path.join(radio_data_path, "output_gece_data.npy")
output_file = os.path.join(radio_data_path, "output_vBvvB_data.npy")

dataset = AntennaDataset(
    input_data_file,
    input_meta_file,
    antenna_pos_file,
    output_meta_file,
    output_file,
    mmap_mode=memmap_mode,
    one_shower=33,
    #percentage=0.01,
)


In [ ]:
def filter_plot(trace):
    tstep = 1e-9
    n_samples = 256
    ff = np.fft.rfftfreq(n_samples, tstep)  # frequencies in Hz
    print(ff.shape)
    tt = tstep * np.arange(n_samples)
    tt *= 1e9  #  time in ns
    spec = np.fft.rfft(trace, axis=-2)
    window = np.zeros(len(ff))
    window[(ff >= 80* 1e6) & (ff <= 200 * 1e6)] = 1
    filtered_spec = np.array(
        [spec[..., 0] * window, spec[..., 1] * window]
    )
    filt = np.fft.irfft(filtered_spec, n_samples, axis=-1)
    plt.plot(filtered_spec[:,0])
    plt.plot(filtered_spec[:,1])

In [ ]:
def process_trace(trace, outp_meta):
    result = np.zeros((256, 4))
    result[:,0] = outp_meta[0] + np.arange(256)*1e-9
    result[:,1:trace.shape[1]+1] = trace
    return result

In [ ]:
def get_fluences(pulses,pos_array, outp_meta):
    energy_fluences = np.zeros((len(pulses), 3))
    for index in range(len(pulses)):
        trace_vB = pulses[index]  # 0,1,2,3: t, vxB, vxvxB, v
        trace_vB = process_trace(trace_vB, outp_meta[0])
        ef = energy_fluence.calculate_energy_fluence_vector(
            trace_vB[:, 1:], trace_vB[:, 0], remove_noise=True
        )
        energy_fluences[index] = ef
    x_pos, y_pos = pos_array[:, 0], pos_array[:, 1]
    if len(energy_fluences.shape) == 1:
        energy_fluences = np.array([energy_fluences]).T    
    return x_pos, y_pos, energy_fluences

In [ ]:
def plot_interpolation(x_pos, y_pos, energy_fluences, hack = True, interp=True, mark_antennas=None, text=None): 
    if hack:
        for index in range(len(energy_fluences)):
            if index % 8 == 0:
                energy_fluences[index] = (
                    energy_fluences[index + 4] + energy_fluences[index + 5]
                ) / 2
            if index % 8 == 2:
                energy_fluences[index] = (
                    energy_fluences[index - 1] + energy_fluences[index + 1]
                ) / 2
    
    
    fig, ax = plt.subplots(
            1,
            4,
            gridspec_kw={"width_ratios": [40, 1, 40, 1]},
            figsize=[int(2.66 * fnt_size), fnt_size],
        )
    if energy_fluences.ndim == 1:
        energy_fluences = energy_fluences.reshape((*energy_fluences.shape,1))
    for ii in range(energy_fluences.shape[-1]):
        energy_flu = energy_fluences[:, ii]

        if np.min(energy_flu) == np.max(energy_flu):
            print(np.min(energy_flu))
            continue

        # define positions where to interpolate
        xs = np.linspace(np.nanmin(x_pos), np.nanmax(x_pos), 100)
        ys = np.linspace(np.nanmin(y_pos), np.nanmax(y_pos), 100)
        xx, yy = np.meshgrid(xs, ys)
        # points within a circle
        in_star = xx**2 + yy**2 <= np.nanmax(x_pos**2 + y_pos**2)
        # interpolated values! but only in the star. outsite set to nan
        if interp:    
            interp_func = intp.Rbf(
                x_pos,
                y_pos,
                energy_flu,
                smooth=0,
                function="quintic",
            )
            fp_interp = np.where(in_star, interp_func(xx, yy), np.nan)
        else:
            fp_interp = energy_flu.reshape((100, 100))
            fp_interp = np.where(in_star, fp_interp, np.nan)
          
        cmap = "inferno"  # set the colormap
        # with vmin/vmax control that both
        # pcolormesh and scatter use the same colorscale
        pcm = ax[2*ii + 0].pcolormesh(
            xx,
            yy,
            fp_interp,
            vmin=np.percentile(energy_flu, 1),
            vmax=np.percentile(energy_flu, 99),
            cmap=cmap,
            shading="gouraud",
        )  # use shading="gouraud" to make it smoother
        _ = ax[2*ii + 0].scatter(
            x_pos,
            y_pos,
            edgecolor="w",
            facecolor="none",
            s=5.0,
            lw=1.0,
        )
        cbi = fig.colorbar(pcm, pad=0.02, cax=ax[2*ii + 1])
        cbi.set_label(r"Energy Fluence $f$ / eV$\,$m$^{-2}$", fontsize=2*fnt_size)

        ax[2*ii + 0].set_ylabel("vvxxB (m)", fontsize=2*fnt_size)
        ax[2*ii + 0].set_xlabel("vxB (m)", fontsize=2*fnt_size)
        ax[2*ii + 0].set_facecolor("black")
        ax[2*ii + 0].set_aspect(1)
        ax[2*ii + 0].set_xlim(np.min(xs), np.max(xs))
        ax[2*ii + 0].set_ylim(np.min(ys), np.max(ys))
        if text is not None:
            ax[2*ii + 0].set_title(f"{text[ii]}", fontsize=2*fnt_size)
        else:
            ax[2*ii + 0].set_title(f"{ii}", fontsize=2*fnt_size)
        print("vmin = ", np.amin(energy_flu))
        print("vmax = ", np.amax(energy_flu))
        ax[2*ii + 0].tick_params(labelsize=2*fnt_size)
        ax[2*ii + 1].tick_params(labelsize=2*fnt_size)
    if mark_antennas is not None:
        _ = ax[2*ii + 0].scatter(
            x_pos[mark_antennas],
            y_pos[mark_antennas],
            edgecolor="r",
            facecolor="none",
            s=500.0,
            lw=5.0,
        ) 
        for i in mark_antennas:
            _ = ax[2*ii + 0].annotate(
                str(i),
                (x_pos[i]+10,y_pos[i]+10), 
                color='green',
                size='large',
        ) 
    plt.tight_layout()
    plt.savefig("fluence_maps.pdf", format="pdf")
    plt.show()  

In [ ]:
def plot_antenna_pulse(antenna_number, outp_data,outm_data, label='blah'):
    assert 1 <= antenna_number <= 240
    assert outm_data[antenna_number - 1, 0] == outm_data[antenna_number -1, 1]
    steps = outm_data[antenna_number -1, 0] + np.arange(0, 256)
    plt.plot(steps, outp_data[antenna_number-1, :, 0], label=label+'_vB')
    plt.plot(steps, outp_data[antenna_number-1, :, 1], label=label+'_vvB')
    plt.xlabel("t (ns)")
    plt.legend()
    plt.show()
    
def plot_all_antennas(outp_datas):
    #plt.violinplot(outp_data[:, :, 0]);
    #plt.show();
    #plt.violinplot(outp_data[:, :, 1]);
    #plt.show();
    fig, axes = plt.subplot_mosaic("AB", sharex=True, sharey=True, figsize=[int(fnt_size), fnt_size/3.33])
    for outp_data in outp_datas:
        #plt.boxplot(outp_data[:, :, 0], showfliers=False, meanline=True);
        axes["A"].plot(np.arange(1, 257), np.median(outp_data[:, :, 0], axis=0))
        axes["A"].fill_between(np.arange(1, 257), np.percentile(outp_data[:, :, 0], 25, axis=0), np.percentile(outp_data[:, :, 0], 75, axis=0), alpha=0.2)
        axes["A"].set_xlabel("t (ns)")
        axes["A"].set_ylabel("E (V/m)")
    for outp_data in outp_datas:
        #plt.boxplot(outp_data[:, :, 1], showfliers=False, meanline=True);
        axes["B"].plot(np.arange(1, 257), np.median(outp_data[:, :, 1], axis=0))
        axes["B"].fill_between(np.arange(1, 257), np.percentile(outp_data[:, :, 1], 25, axis=0), np.percentile(outp_data[:, :, 1], 75, axis=0), alpha=0.2)
        axes["B"].set_xlabel("t (ns)")
        axes["B"].set_ylabel("E (V/m)")
    plt.savefig("all_antennas.pdf", format='pdf')
    plt.show();


In [ ]:
def box_plot(data, index=1):

    _ = plt.figure(figsize=(fnt_size, fnt_size/2.33), )
    plt.boxplot(data[:, :, index], showfliers=False, meanline=True);
    plt.xticks()
    plt.savefig("boxplot.pdf")
    plt.show()

In [ ]:
def plot_antenna_pulse2(antenna_number, outp_data1, outp_data2,outm_data):
    assert 1 <= antenna_number <= 240
    assert outm_data[antenna_number - 1, 0] == outm_data[antenna_number -1, 1]
    steps = outm_data[antenna_number -1, 0] + np.arange(0, 256)
    fig, ax = plt.subplot_mosaic("AB", figsize=[int(fnt_size), fnt_size/3.33])
    ax["A"].plot(steps, outp_data1[antenna_number-1, :, 0], label='NN')
    ax["A"].plot(steps, outp_data2[antenna_number-1, :, 0], label='CoREAS')
    ax["A"].set_title(f"Geomagnetic Effect for Antenna {antenna_number}")
    ax["B"].plot(steps, outp_data1[antenna_number-1, :, 1], label='NN')
    ax["B"].plot(steps, outp_data2[antenna_number-1, :, 1], label='CoREAS')
    ax["B"].set_title(f"Charge Excess for Antenna {antenna_number}")
    ax["A"].set_xlabel("t (ns)");    ax["B"].set_xlabel("t (ns)")
    ax["A"].set_ylabel("E (V/m)");    ax["B"].set_ylabel("E (V/m)")
    ax["A"].legend();     ax["B"].legend()
    fig.tight_layout()
    return fig


In [ ]:
from radioNN.data.transforms import sph2cart, cart2sph
sph2cart(cart2sph(np.array([[1, 0, 0], [0, 1, 0], [-1, 0, 0], [0, -1, 0]], dtype=float)))

In [ ]:
inp_data_real, inp_meta_real, antenna_pos_real, outp_meta_real, outp_data_real = dataset.data_of_single_shower(16)
#plot_interpolation(*get_fluences(outp_data_real, antenna_pos_real*250, outp_meta_real))

In [ ]:
#box_plot(outp_data_real)

In [ ]:
box_plot(outp_data_real, index=0)

In [ ]:
plot_all_antennas([outp_data_real])

In [ ]:
one_shower = 17
print(f"Use shower {one_shower}")
process = NetworkProcess(model_class=AntennaNetworkSkipFC,
                       # one_shower=one_shower, 
                        percentage=100,
                         batch_size=8, wb=False,
                         )
num_epochs = 500
#print(process.model)

In [ ]:
#process.full_training(num_epochs)
#process.model = torch.load("/home/pranav/MEGA/work-stuff/radio_nn/runs/2309Sep08Fri_172951/SavedModel", map_location=torch.device('cpu'))
#process.model = torch.load("/home/pranav/MEGA/work-stuff/radio_nn/runs/2309Sep26Tue_161654/SavedModel", map_location=torch.device('cpu'))
process.model = torch.load("runs/2503Mar25Tue_100818/SavedModel", map_location=torch.device('cpu'))


process.model.eval()

In [ ]:
outp_meta_sim, outp_data_sim = process.pred_one_shower(17)
inp_data_real, inp_meta_real, antenna_pos_real, outp_meta_real, outp_data_real = dataset.data_of_single_shower(17)
plot_antenna_pulse2(12, outp_data_sim, outp_data_real, outp_meta_real);

In [ ]:
plot_all_antennas([outp_data_real, outp_data_sim])

In [ ]:
plot_all_antennas([outp_data_real])
plot_all_antennas([outp_data_sim])

In [ ]:
print(f"Number of Parameters: {sum(p.numel() for p in process.model.parameters())}")

In [ ]:
print(f"Number of Trainable Parameters: {sum(p.numel() for p in process.model.parameters() if p.requires_grad)}")

In [ ]:
from radioNN.data.transforms import sph2cart, cart2sph
def check_shower(shower, antenna=None):
    print(shower)
    sim = process.pred_one_shower(shower)[1]
    data = dataset.data_of_single_shower(shower)
    pos, meta, real = data[2], data[3], data[4]
    pos = sph2cart(pos)
    if antenna is None:
        antenna = np.random.choice(np.arange(240))
    #plot_antenna_pulse2(antenna, sim, real, meta);
    #plot_all_antennas([sim])
    #plot_all_antennas([real])
    plot_interpolation(*get_fluences(real, pos, meta))
    plot_interpolation(*get_fluences(sim, pos, meta))
    error = ((real-sim)**2).mean(axis=(1,2))
    plot_interpolation(pos[:,0], pos[:,1],error)
    #plot_interpolation(*get_fluences(process.pred_one_shower_entire_array(shower)[1], pos*250, meta), interp=False, hack=False)
    print("--------------------------------------")

In [ ]:
check_shower(33, antenna=14)

In [ ]:
plt.boxplot(process.dataset.return_data()[1].T, showfliers=False, meanline=True);

In [ ]:
plt.boxplot(process.dataset.return_data()[1].T, showfliers=False, meanline=True);

In [ ]:
for sh in np.random.choice(np.unique(process.dataset.indices//240), size=5):
    pass
    #check_shower(sh)

In [ ]:
for sh in np.random.choice(np.arange(58266), size=5):
    pass
    #check_shower(sh)


In [ ]:
??dataset.data_of_single_shower

In [ ]:
sim = process.pred_one_shower(27575)[1]
data = dataset.data_of_single_shower(27575)
pos, meta, real = data[2], data[3], data[4]
pos = sph2cart(pos)

In [ ]:
plot_antenna_pulse2(14, sim, real, meta);

In [ ]:
plot_antenna_pulse2(90, sim, real, meta);

In [ ]:
plot_antenna_pulse2(200, sim, real, meta)

In [ ]:
box_plot(real);


In [ ]:
box_plot(sim);

In [ ]:
mask = np.logical_and(30 <= np.fft.rfftfreq(256, 1e-9)/1e6, np.fft.rfftfreq(256, 1e-9)/1e6 <=80)

In [ ]:
plt.plot((np.fft.rfftfreq(256, 1e-9)/1e6)[mask],np.fft.rfft(real[14,:,0]).real[mask], marker='.')
plt.plot((np.fft.rfftfreq(256, 1e-9)/1e6)[mask],np.fft.rfft(real[14,:,0]).imag[mask], marker='.')

In [ ]:
plt.plot((np.fft.rfftfreq(256, 1e-9)/1e6)[mask],np.abs(np.fft.rfft(real[14,:,0]))[mask], marker='.')


In [ ]:
plt.plot((np.fft.rfftfreq(256, 1e-9)/1e6)[mask],((np.angle(np.fft.rfft(real[14,:,0]))+np.pi/2)%np.pi)[mask]/np.pi, marker='.')
plt.ylabel(r"$\theta/\pi$")
plt.xlabel("$f$")

In [ ]:
plt.boxplot(np.abs(np.fft.rfft(real[:,:,0], axis=1)));
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])
plt.yscale("log")
plt.ylabel("power spectrum")
plt.xlabel("freq")
plt.title("Real Power spectrum distribution")
plt.savefig("realspec.pdf", format='pdf')

In [ ]:
plt.boxplot(np.abs(np.fft.rfft(sim[:,:,0], axis=1)));
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])
plt.yscale("log")
plt.ylabel("power spectrum")
plt.xlabel("freq")
plt.title("Generated Power spectrum distribution")
plt.savefig("simspec.pdf", format='pdf')

In [ ]:
plt.boxplot(np.real(np.fft.rfft(real[:,:,0], axis=1)),showfliers=False);
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])

In [ ]:
plt.boxplot(np.imag(np.fft.rfft(real[:,:,0], axis=1)),showfliers=False , meanline=True);
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])

In [ ]:
plt.boxplot(((np.angle(np.fft.rfft(real[:,:,0], axis=1))))/np.pi);
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])
plt.ylabel("phase")
plt.title("Real Phase distribution")
plt.xlabel("freq")

In [ ]:
plt.boxplot(((np.angle(np.fft.rfft(sim[:,:,0], axis=1))))/np.pi);
plt.xlim([np.min(np.arange(129)[mask]), np.max(np.arange(129)[mask])+2])
plt.ylabel("phase")
plt.title("Generated Phase distribution")
plt.xlabel("freq")

In [ ]:
if True:
    from scipy import signal as sg
    tdata = real[45,:,0]
    print(tdata.shape)
    hdata = sg.hilbert(tdata)
    plt.plot(np.abs(hdata));plt.show()
    plt.plot(np.angle(hdata));plt.show()
    plt.plot(tdata);plt.show()
    phase = (np.angle(np.fft.rfft(tdata))[mask])
    spec = np.abs(np.fft.rfft(tdata))[mask]
    plt.plot(spec) ; plt.show()
    plt.plot(phase); plt.show()
    data = np.zeros(129, dtype=np.complex256)
    data[mask] = spec * np.exp(1j*phase)
    plt.plot(np.fft.irfft(data), label="reverse")
    plt.plot(tdata, label="real")
    plt.legend()
    plt.show()


In [ ]:
np.fft.rfft(real, axis=1).T.shape

In [ ]:
real.shape

In [ ]:
np.concatenate([np.fft.rfft(real.T, axis=1)[:,mask],np.fft.rfft(real.T, axis=1)[:,mask]], axis=0).shape

In [ ]:
26387

In [ ]:
def plot_polarity(pos, polarity, radius):  
    plt.scatter(pos[:, 0][polarity[:,0]], pos[:, 1][polarity[:,0]], marker='.', label='+')
    plt.scatter(pos[:, 0][~polarity[:,0]], pos[:, 1][~polarity[:,0]], marker='.', label='-')
    plt.gca().add_artist(plt.Circle((0,0), radius,  alpha=0.1))
    plt.legend()
    plt.show()

def check_shower(shower, antenna=None):
    print(shower)
    sim = process.pred_one_shower(shower)[1]
    data = dataset.data_of_single_shower(shower)
    pos, meta, real = data[2], data[3], data[4]
    pos = sph2cart(pos)
    if antenna is None:
        antenna = np.random.choice(np.arange(240))
    else:
        for i in antenna:
            plot_antenna_pulse2(i, sim, real, meta);

    #plot_all_antennas([sim])
    #plot_all_antennas([real])
    plot_interpolation(*get_fluences(real, pos, meta), text=["Geomagnetic Effect","Charge Excess"])
    plot_interpolation(*get_fluences(sim, pos, meta), text=["Geomagnetic Effect","Charge Excess"])
    #error = ((real-sim)**2).mean(axis=(1,2))
    #plot_interpolation(pos[:,0], pos[:,1],error)
    

    #plot_interpolation(*get_fluences(process.pred_one_shower_entire_array(shower)[1], pos*250, meta), interp=False, hack=False)
    print("--------------------------------------")
    polarity = ((real-np.abs(real)**2).mean(axis=(1,2))) > (np.abs(real)**2).mean(axis=(1,2))
    #plot_interpolation(pos[:,0], pos[:,1],np.where(polarity,1, -1), interp=True )

    #plot_polarity(pos, polarity, 144)

    

    pulse_max = np.max(real, axis=1)
    pulse_min = np.min(real, axis=1)
    polarity = np.where(np.abs(pulse_max) > np.abs(pulse_min), 1, -1)
    pulse_max = np.max(sim, axis=1)
    pulse_min = np.min(sim, axis=1)
    polarity_sim = np.where(np.abs(pulse_max) > np.abs(pulse_min), 1, -1)
    plot_interpolation(pos[:,0], pos[:,1],polarity, interp=True, mark_antennas=antenna, text=["Geomagnetic Effect","Charge Excess"] )
    plot_interpolation(pos[:,0], pos[:,1],polarity_sim, interp=True, mark_antennas=antenna , text=["Geomagnetic Effect","Charge Excess"])
    plot_interpolation(pos[:,0], pos[:,1],np.log10(np.max(np.abs(real), axis=1)), interp=True, mark_antennas=antenna , text=["Geomagnetic Effect","Charge Excess"])
    plot_interpolation(pos[:,0], pos[:,1],np.log10(np.max(np.abs(sim), axis=1)), interp=True, mark_antennas=antenna , text=["Geomagnetic Effect","Charge Excess"])
    plt.hist(np.log10(np.max(np.abs(real), axis=1)), bins=50, log=True)
    plt.show()
    plt.hist(np.log10(np.max(np.abs(sim), axis=1)), bins=50, log=True)
    plt.show()
    plot_antenna_pulse2(205, sim, real, meta)
    #polarity = np.where(np.abs(pulse_max) < np.abs(pulse_min), True, False)
    #print(polarity.shape)
    #print(polarity.sum()/2)
    #plot_polarity(pos, polarity, 144)
    plot_interpolation(pos[:,0], pos[:,1],polarity, interp=True, text=["Geomagnetic Effect","Charge Excess"] )
    plot_interpolation(pos[:,0], pos[:,1],np.max(np.abs(real), axis=1), interp=True, text=["Geomagnetic Effect","Charge Excess"])

check_shower(26387, antenna=[14, 45, 90])
#check_shower(27575, antenna=[14, 45, 90])

In [ ]:
plot_interpolation( pos[:,0], pos[:,1],np.exp(meta), interp=True , hack=False)

In [ ]:
check_shower(26387, antenna=[14, 45, 90, 157])

In [ ]:
inp_meta_real[3]

In [ ]:
np.arccos(1/inp_meta_real[3])

In [ ]:
inp_meta_real

In [ ]:
inp_data_real[0].shape

In [ ]:
inp_meta_real[1]*700

In [ ]:
dataset.input_data[26387].shape

In [ ]:
dataset.input_data[0].shape

In [ ]:
plt.plot(dataset.input_data[26387][:,6])
plt.vlines(70, 0, np.max(dataset.input_data[26387][:,6]), 'r')

In [ ]:
743038/100

In [ ]:
7.43038e3

In [ ]:
dataset.input_meta[26387][2]

In [ ]:
1.67e5

In [ ]:
dataset.input_data[26387][:,9]

In [ ]:
np.arccos( 1/1.0000069)

In [ ]:
9354*np.arccos( 1/1.00012)

In [ ]:
dataset.input_meta[26387][4]

In [ ]:
def box_plot(data):
    SMALL_SIZE = 14
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 22

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    _ = plt.figure(figsize=(fnt_size, fnt_size/2.33), )
    plt.boxplot(data[:, :, 1], showfliers=False, meanline=True);
    plt.xticks(ticks=np.where(np.arange(256)%10==0)[0], labels=np.where(np.arange(256)%10==0)[0])
    plt.title("All the radio pulses for a single event")
    plt.ylabel("E(V/m)")
    plt.xlabel("t(ns)")
    plt.savefig("boxplot.pdf", format='pdf')
    plt.show()
box_plot(real);

In [ ]:
box_plot(sim);

In [ ]:
np.where(np.arange(256)%10==9)

In [ ]:
?np.where